In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

import multiprocessing as mp

from LP_Action_Liquidity_Parameter_Function import Get_Liquidity_Parameter, Get_Price

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH 500 10'

# LP Action

We import all LP actions and conduct basic cleaning.

In [2]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]
lp_action.dropna(subset = ['AMOUNT0_USD','AMOUNT1_USD'], inplace = True)

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]
lp_action.drop(columns = ['NF_TOKEN_ID'], inplace = True)

# Changing signs of addition and extraction
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price)
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price)

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

print("Number of LP Actions in this pool:",len(lp_action))
lp_action.head()

Number of LP Actions in this pool: 160799


,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 500 10,Increase Liquidity,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,2995.507735,USDC,2995.292327,1.000000,WETH,3437.435736,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
1,USDC-WETH 500 10,Decrease Liquidity,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,2588.187276,USDC,2588.001158,0.931484,WETH,3201.917248,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
2,USDC-WETH 500 10,Increase Liquidity,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,1.000000,USDC,0.999928,0.000146,WETH,0.500964,194990,195010,3398.18281290132699723471887409687042236328125,3404.98563895004917867481708526611328125
3,USDC-WETH 500 10,Increase Liquidity,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,598.469729,USDC,598.426693,2.600000,WETH,8937.332900,186730,195460,3248.661520935142107191495597362518310546875,7777.2237085991100684623233973979949951171875
4,USDC-WETH 500 10,Increase Liquidity,12377308,2021-05-05 23:44:49,0x9f9b987becfe15aca93ce9208cca146f252c8d46,3405.610348,USDC,3405.365449,0.000000,WETH,0.000000,194660,194990,3404.98563895004917867481708526611328125,3519.21893691676132220891304314136505126953125


# Match Withdrawal with Deposition

In [3]:
token_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))
token_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]
token_transfer = token_transfer.sort_values(by = ['TOKENID','BLOCK_NUMBER'])

token_transfer = token_transfer[token_transfer['BLOCK_NUMBER'] <= lp_action['BLOCK_NUMBER'].max()]

token_transfer.head()

,NFT_ADDRESS,BLOCK_TIMESTAMP,BLOCK_NUMBER,NFT_FROM_ADDRESS,NFT_TO_ADDRESS,TOKENID
323756,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-05-04 19:52:36+00:00,12369739,0x0000000000000000000000000000000000000000,0x11e4857bb9993a50c685a79afad4e6f65d518dda,1
391017,0xc36442b4a4522e871399cd717abdd847ab11fe88,2022-12-24 18:49:35+00:00,16256492,0x11e4857bb9993a50c685a79afad4e6f65d518dda,0x50ec05ade8280758e2077fcbc08d878d4aef79c3,1
327263,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-05-04 19:58:13+00:00,12369760,0x0000000000000000000000000000000000000000,0x4bd047ca72fa05f0b89ad08fe5ba5ccdc07dffbf,2
583092,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-05-04 20:07:45+00:00,12369811,0x0000000000000000000000000000000000000000,0xe79057088a15ac8c9c8bec3b90bd9891a1b3af51,3
459181,0xc36442b4a4522e871399cd717abdd847ab11fe88,2021-05-04 20:09:34+00:00,12369821,0x0000000000000000000000000000000000000000,0x4bd047ca72fa05f0b89ad08fe5ba5ccdc07dffbf,4


Find the initial address and the final (current) address of the token.

In [4]:
initial = token_transfer.loc[token_transfer['NFT_FROM_ADDRESS'] == '0x0000000000000000000000000000000000000000',['TOKENID','BLOCK_NUMBER','NFT_TO_ADDRESS']]
initial['EVENT_NAME'] = 'Increase Liquidity'

final = token_transfer.groupby(['TOKENID'])[['TOKENID','BLOCK_NUMBER','NFT_TO_ADDRESS']].tail(1)
final['EVENT_NAME'] = 'Decrease Liquidity'

token_transfer = pd.concat([initial,final], axis = 0)
token_transfer.sort_values(by = ['TOKENID','BLOCK_NUMBER'], inplace = True)
token_transfer.rename(columns = {'NFT_TO_ADDRESS':'SENDER'}, inplace = True)

token_transfer.head()

,TOKENID,BLOCK_NUMBER,SENDER,EVENT_NAME
323756,1,12369739,0x11e4857bb9993a50c685a79afad4e6f65d518dda,Increase Liquidity
391017,1,16256492,0x50ec05ade8280758e2077fcbc08d878d4aef79c3,Decrease Liquidity
327263,2,12369760,0x4bd047ca72fa05f0b89ad08fe5ba5ccdc07dffbf,Increase Liquidity
327263,2,12369760,0x4bd047ca72fa05f0b89ad08fe5ba5ccdc07dffbf,Decrease Liquidity
583092,3,12369811,0xe79057088a15ac8c9c8bec3b90bd9891a1b3af51,Increase Liquidity


Now, we are able to match every withdrawal with every deposition.

# Add Liquidity Parameter

In [ ]:
with mp.Pool(10) as pool:
    result = pool.imap(Get_Liquidity_Parameter, lp_action[['AMOUNT1_ADJUSTED','AMOUNT0_ADJUSTED','PRICE_LOWER','PRICE_UPPER']].iterrows(), chunksize = 1)
    output = [x for x in result]

lp_action['L'] = output

In [ ]:
lp_action.to_csv(os.path.join(data_dir, "uniswapv3_lp_action_LP", pool_name + "_LP.csv"), index=False)